# Google Drive Preliminaries

# Import libraries

In [1]:
import numpy as np
from numpy.random import RandomState
import numpy.ma as ma

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline

import h5py
import ot
from numpy.random import Generator, PCG64
from sklearn import metrics
import itertools

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

In [2]:
sigAliasList    = ['sig_A', 'sig_h0', 'sig_hch', 'sig_LQ']
sigFilenameList = ['Ato4l_lepFilter_13TeV_filtered.h5', 'hToTauTau_13TeV_PU20_filtered.h5', 'hChToTauNu_13TeV_PU20_filtered.h5', 'leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5']

In [3]:
#-- Set base directory and data directory path --#
basePath   = '/Users/hanchengli/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/'
dataPath   = 'Data/'

bkgPath    = basePath+dataPath+'background_for_training.h5'
sigPathList = []
for x in sigFilenameList:
  sigPathList.append(basePath+dataPath+x)

# Functions

In [4]:
%cd ~/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/functions
%run centralFunctions.ipynb

/Users/hanchengli/Library/CloudStorage/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/functions


# Loading Data

In [5]:
dataDict = {}
dataDict['bkg'] = h5py.File(bkgPath, 'r')

for i in range(len(sigAliasList)):
  alias   = sigAliasList[i]
  sigPath = sigPathList[i]
  dataDict[alias] = h5py.File(sigPath, 'r')

In [6]:
bkg_data = dataDict['bkg']['Particles'][:, :, 0:3]
sig_data = {}

for alias in sigAliasList:
  sig_data[alias] = dataDict[alias]['Particles'][:, :, 0:3]

# kNN using distance matrix with 3d ground space

In [7]:
nEvents = 1000
random_state = Generator(PCG64(1))
OTSCHEME = {}
OTSCHEME['normPT'] = True
OTSCHEME['balanced'] = True
OTSCHEME['noZeroPad'] = False
OTSCHEME['individualOT'] = False

In [8]:
events = {}

events['bkg'] = randomDataSample(bkg_data, nEvents, random_state)

neighbor_list = list(range(5, 400,10))
avg_aucs = []
std_aucs = []
avg_ks = []
std_ks = []

i = 0

for alias in sigAliasList:
    events[alias] = randomDataSample(sig_data[alias], nEvents, random_state)

del sig_data, bkg_data

for alias in sigAliasList:
    np.random.seed(i)
    permutation = np.random.permutation(nEvents * 2)
    
    event_list = np.concatenate((events['bkg'], events[alias]))
    event_labels = np.asarray([0] * nEvents + [1] * nEvents)
    event_list = event_list[permutation]
    event_labels = event_labels[permutation]
    
    distance_matrix = calcOTDistance(event_list, event_list, OTSCHEME, '2D', Matrix = True)
    
    avg_auc, std_auc, avg_k, std_k = kNN_cross_validation(distance_matrix, event_labels, neighbor_list, k_fold=5)
    print(avg_auc, std_auc, avg_k, std_k)
    avg_aucs.append(avg_auc)
    std_aucs.append(std_auc)
    avg_ks.append(avg_k)
    std_ks.append(std_k)
    i += 1

Fitting Models: 100%|██████████| 40/40 [00:01<00:00, 31.08it/s]


0.6947582733388014 0.017948357453657943 37.0 23.15167380558045


Fitting Models: 100%|██████████| 40/40 [00:01<00:00, 22.86it/s]


0.669764010403308 0.011783244458465292 145.0 86.48699324175861


Fitting Models: 100%|██████████| 40/40 [00:01<00:00, 23.93it/s]


0.8103431594284152 0.016734256523563844 161.0 85.46344247688599


Fitting Models: 100%|██████████| 40/40 [00:01<00:00, 23.26it/s]


0.7905600332483893 0.025314221471884484 163.0 24.819347291981714
